In [1]:
#Import useful functions
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.utils.multiclass import unique_labels
import itertools
from scipy import misc
import math
import os
import glob
import xlrd
import xlwt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from PIL import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import pylab as plt
import numpy as np
from scipy.signal import argrelextrema
from IPython.display import Image
import matplotlib.patches as patches
import matplotlib as matplotlib
import xlsxwriter
from xlrd import open_workbook
from xlutils.copy import copy
from sklearn.model_selection import train_test_split
from datetime import datetime
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd

In [2]:
foldername1 = '/Users/JoyThompson/Desktop/Contract Expiration Dates JoyT Dec 23.xlsx'
data=pd.read_excel(foldername1, sheetname='Consolidated')
bill=pd.read_excel(foldername1, sheetname='Billing Auto')
bill=bill.set_index('Client Name')
month='2019-12-01'
kpipd=pd.read_excel(foldername1, sheetname='Retention KPI')
kpipd=kpipd.set_index('Month')
pods=pd.read_excel(foldername1, sheetname='Pod V Pod')
pods=pods.set_index('Pod')
amazon=pd.read_excel(foldername1, sheetname='SC V VC')
amazon=amazon.set_index('Amazon Store')
poddata=pd.read_excel(foldername1, sheetname='Pod Data')
poddata=poddata.set_index('Pod')
amazon=pd.read_excel(foldername1, sheetname='SC V VC')
accdata=pd.read_excel(foldername1, sheetname='Account Type Data')
accdata=accdata.set_index('Account')
retentiondata=pd.read_excel(foldername1, sheetname='retentiondata')

In [3]:
podS=data[data['Pod'] == "Spencer's Pod"]
podB=data[data['Pod'] == "Brian's Pod"]
podOB=data[data['Pod'] == "On - Boarding"]
Hybrid=data[data['VC, SC, or Hybrid'] == "Hybrid"]
SC=data[data['VC, SC, or Hybrid'] == "SC"]
VC=data[data['VC, SC, or Hybrid'] == "VC"]

In [4]:
origc=newc=outc=0
dropclient=[]
for ind in data.index:
    new=data.loc[ind, 'New Client']
    out=data.loc[ind, 'Opt Out Date']
    if new == 'Yes':
        newc=newc+1
    else:
        origc=1+origc
    if str(out)[0]=='N':
        outc=outc+1
    else:
        dropclient.append(ind)
print(newc)      
inddata=len(retentiondata)-13
indd=inddata+1
inddataq=len(retentiondata)-3
inddq=inddataq+1

origcq=retentiondata.loc[inddataq,'# of Full Service Active Clients at Month End']      
origc=retentiondata.loc[inddata,'# of Full Service Active Clients at Month End']  
newc=newc+sum(retentiondata.loc[indd:,'# of Full Service New Clients Added'])
newcq=newc+sum(retentiondata.loc[inddq:,'# of Full Service New Clients Added'])
retention=((outc-newc)/origc)
print(outc)
print(newc)
print(retentiondata.loc[inddata,'Month'])
print(origc)
if month in ['2019-12-01','2019-03-01','2019-06-01', '2019-09-01']:
    retentionq=((outc-newcq)/origcq)
else:
    retentionq=data.loc[-1, 'Quarterly Rolling Retention Rate']

3
44
43
2018-11-01 00:00:00
25


In [5]:
def retentioncalc(data, retentiondata):
    origc=newc=outc=0
    dropclient=[]
    for ind in data.index:
        new=data.loc[ind, 'New Client']
        out=data.loc[ind, 'Opt Out Date']
        if new == 'Yes':
            newc=newc+1
        else:
            origc=1+origc
        if str(out)[0]=='N':
            outc=outc+1
        else:
            dropclient.append(ind)
   
    return dropclient, outc, newc, origc

In [6]:
dropclientS, outcS, newcS, origcS = retentioncalc(podS, poddata)
dropclientB, outcB, newcB, origcB = retentioncalc(podB, poddata)
dropclientOB,outcOB, newcOB, origcOB = retentioncalc(podOB, poddata)
dropclientH, outcH, newcH, origcH = retentioncalc(Hybrid, accdata)
dropclientSC, outcSC, newcSC, origcSC = retentioncalc(SC, accdata)
dropclientVC, outcVC, newcVC, origcVC = retentioncalc(VC, accdata)

In [7]:
currentcl=data.drop(dropclient)
currentclients=currentcl.set_index('Client Name')

In [8]:
def renewcalc(data, dropclient, origc):    
    datadrop=data.loc[dropclient,:]
    if len(datadrop)>0:
        for ind in datadrop.index:
            start=datadrop.loc[ind, 'Initial Start Date']
            end=datadrop.loc[ind, 'Contract End Date']
            lifetime=str(end-start)
            datadrop.loc[ind, 'Client Lifetime']=float(lifetime.split(' ')[0])
        averagelt=np.mean(datadrop['Client Lifetime'])   
    else:
        averagelt=365
    today=str(datetime.today())[0:6]
    renew=drop=optout=0
    for ind in data.index:
        end=data.loc[ind, 'Contract End Date']
        out=data.loc[ind, 'Opt Out Date']
        end=str(end)[0:6]
        if end == today:
            if str(out)[0]=='N':
                renew=renew+1
            else:
                drop=drop+1
        if str(out)[0]!='N':
            if end != today:
                optout=optout+1
    if float(renew+drop)>0:
        renewper=(renew/(renew+drop))
    else:
        renewper=1
    optper=(optout/origc)
    return averagelt, renewper, optper, datadrop


In [9]:
averagelt, renewper, optper, datadrop = renewcalc(data,dropclient, origc)
averageltS, renewperS, optperS, datadropS = renewcalc(podS, dropclientS, origcS)
averageltB, renewperB, optperB, datadropB = renewcalc(podB, dropclientB, origcB)
averageltOB, renewperOB, optperOB, datadropOB = renewcalc(podOB, dropclientOB, origcOB)
averageltH, renewperH, optperH, datadropH = renewcalc(Hybrid, dropclientH, origcH)
averageltSC, renewperSC, optperSC, datadropSC = renewcalc(SC, dropclientSC, origcSC)
averageltVC, renewperVC, optperVC, datadropVC = renewcalc(VC, dropclientVC, origcVC)

In [10]:

poddata=poddata.append({'Pod': 'Spencer', 'Month': month, 'Original Number of Clients': origcS,
                       'End Number of Clients':outcS, 'New Clients':newcS, 'Clients Opt Out':optperS,
                       'Clients Renew':renewperS, 'Average Lifetime':averageltS}, ignore_index=True)
poddata=poddata.append({'Pod': 'Brian', 'Month': month, 'Original Number of Clients': origcB,
                       'End Number of Clients':outcB, 'New Clients':newcB,'Clients Opt Out':optperB,
                       'Clients Renew':renewperB, 'Average Lifetime':averageltB}, ignore_index=True)
poddata=poddata.append({'Pod': 'On - Boarding', 'Month': month, 'Original Number of Clients': origcOB,
                       'End Number of Clients':outcOB, 'New Clients':newcOB,'Clients Opt Out':optperOB,
                       'Clients Renew':renewperOB, 'Average Lifetime':averageltOB}, ignore_index=True)
accdata=accdata.append({'Account': 'Seller Central', 'Month': month, 'Original Number of Clients': origcSC,
                       'End Number of Clients':outcSC, 'New Clients':newcSC, 'Clients Opt Out':optperSC,
                       'Clients Renew':renewperSC, 'Average Lifetime':averageltSC}, ignore_index=True)
accdata=accdata.append({'Account': 'Vendor Central', 'Month': month, 'Original Number of Clients': origcVC,
                       'End Number of Clients':outcVC, 'New Clients':newcVC, 'Clients Opt Out':optperVC,
                       'Clients Renew':renewperVC, 'Average Lifetime':averageltVC}, ignore_index=True)
accdata=accdata.append({'Account': 'Hybrid', 'Month': month, 'Original Number of Clients': origcH,
                       'End Number of Clients':outcH, 'New Clients':newcH, 'Clients Opt Out':optperH,
                       'Clients Renew':renewperH, 'Average Lifetime':averageltH}, ignore_index=True)
retentiondata=retentiondata.append({'Month':month, '# of Full Service Active Clients at Month End': outc,
                                   '# of Full Service New Clients Added':newc,
                                    '# of Cancelled Full Service Clients': optper, "# of Clients Renewed": renewper}, ignore_index=True)


In [11]:
billav=bill.loc['Average', month]
ltval=float(billav)*float(averagelt/30.5)

In [12]:
kpipd.loc[month, 'Average Monthly Client Revenue']=np.round(ltval, decimals=2)
kpipd.loc[month, 'Average Customer Lifetime Value']=np.round(billav, decimals=2)
kpipd.loc[month, 'Annual Rolling Retention Rate']=retention
kpipd.loc[month, 'Quarterly Rolling Retention Rate']=retentionq
kpipd.loc[month, 'Average Client Term']=averagelt
kpipd.loc[month, 'Monthly Client Opt Out Rate']=optper
kpipd.loc[month, 'Monthly Client Renewal Rate']=renewper

In [13]:
podlt=[averageltS, averageltB, averageltOB]
podren=[renewperS, renewperB, renewperOB]
podopt=[optperS, optperB, optperOB]
podlist = ["Spencer's Pod", "Brian's Pod", "On - Boarding"]


In [14]:
count=0
for ind in pods.index:

    lifetime=podlt[count]
    renew=podren[count]
    optout=podopt[count]
    pod=podlist[count]
    pods.loc[pod, 'Average Client Term']=lifetime
    pods.loc[pod, 'Client Renewal Rate']=renew
    pods.loc[pod, 'Client Opt Out Rate']=optout
    count=count+1
print(pods)

               Average Client Term  Client Renewal Rate  Client Opt Out Rate
Pod                                                                         
Spencer's Pod           364.333333             0.333333             0.047619
Brian's Pod             365.000000             1.000000             0.000000
On - Boarding            91.000000             0.500000             0.000000


In [15]:
amlt=[averageltSC, averageltVC, averageltH]
amren=[renewperSC, renewperVC, renewperH]
amopt=[optperSC, optperVC, optperH]
storelist = ["Seller Central", "Vendor Central", "Hybrid"]

In [16]:
amazon=amazon.set_index('Amazon Store')
count=0
for ind in amazon.index:

    lifetime=amlt[count]
    renew=amren[count]
    optout=amopt[count]
    store=storelist[count]
    amazon.loc[store, 'Average Client Term']=lifetime
    amazon.loc[store, 'Client Renewal Rate']=renew
    amazon.loc[store, 'Client Opt Out Rate']=optout
    count=count+1

In [17]:
print(amazon)

                Average Client Term  Client Renewal Rate  Client Opt Out Rate
Amazon Store                                                                 
Seller Central                296.0                  0.7             0.026316
Vendor Central                365.0                  1.0             0.000000
Hybrid                        365.0                  1.0             0.000000


In [19]:
filenamefinal= '/Users/JoyThompson/Desktop/RetentionKPIAutomatedV7.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
kpipd.to_excel(writer, sheet_name='Retention KPI')
currentclients.to_excel(writer, sheet_name='Current CK Clients')
pods.to_excel(writer, sheet_name='Pod vs Pod')
retentiondata.to_excel(writer, sheet_name='Retention D')

amazon.to_excel(writer, sheet_name='Amazon Account Comparison')
poddata.to_excel(writer, sheet_name='Pod vs Pod Data')
accdata.to_excel(writer, sheet_name='Amazon Account Comparison Data')
writer.save()